In [14]:
import os
import sys
from dotenv import load_dotenv

from sqlalchemy import create_engine

import pandas as pd


from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [15]:
load_dotenv()
database_url = os.getenv('MYSQL_ENGINE_URL')

In [16]:
engine = create_engine(database_url)

In [17]:
query = "SELECT * FROM hotel_booking LIMIT 200;"

In [37]:
data = pd.read_sql(query, engine)
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [38]:
# Dropping the features
data = data.drop(['reservation_status',
              'reservation_status_date',
              'assigned_room_type', 
              'arrival_date_year', 
              'country', 
              'agent', 
              'company', 
              'name', 
              'email', 
              'phone-number', 
              'credit_card'], 
              axis=1)

In [39]:
data.shape

(200, 25)

In [40]:
data.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
dtype: int64

In [41]:
# Analyze noisy data
noisy_data = {
    'adr':      data[data['adr'] < 0],
    'adults':   data[data['adults'] == 0],
    'children': data[data['children'] == 10],
    'babies':   data[data['babies'] == 10],
}

noisy_data_count = {key: len(value) for key, value in noisy_data.items()}
noisy_data_count

{'adr': 0, 'adults': 0, 'children': 0, 'babies': 0}

In [42]:
# for preprocessing
one_hot_cols = ['hotel', 'meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'deposit_type', 'customer_type']
label_cols = ['arrival_date_month']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [43]:
oh_enc = OneHotEncoder(sparse=False, drop='first')

label_enc = LabelEncoder()

In [50]:
one_hot_cols

['hotel',
 'meal',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'deposit_type',
 'customer_type']

In [48]:
encoded_arr = oh_enc.fit_transform(data[one_hot_cols])
encoded_df = pd.DataFrame(encoded_arr, columns=oh_enc.get_feature_names_out(one_hot_cols))


ValueError: Shape of passed values is (200, 1), indices imply (200, 23)

In [ ]:
def get_preprocessing_functions(self) -> Pipeline: #tuple:
    try:
        # Fetch schema config
        onehot_encoding_columns = one_hot_cols
        label_encoding_columns = label_cols
        scaling_columns = self._schema_config.get('scaling_columns', [])

        # Define individual transformer functions
        def label_encoding_function(data: pd.DataFrame) -> pd.DataFrame:
            month_order = months
            columns = label_encoding_columns if isinstance(label_encoding_columns, list) else [label_encoding_columns]
            for col in columns:
                data[col] = data[col].apply(lambda x: month_order.index(x) + 1)
            return data

        def onehot_encoding_function(data: pd.DataFrame) -> pd.DataFrame:
            data = pd.get_dummies(data, columns=onehot_encoding_columns, drop_first=True)
            data = data.astype(int)
            return data
        
        def scaling_function(data: pd.DataFrame) -> pd.DataFrame:
            data[scaling_columns] = scaler.fit_transform(data[scaling_columns])
            return data

        # Initialize transformers
        label_encoder = FunctionTransformer(label_encoding_function)
        onehot_encoder = FunctionTransformer(onehot_encoding_function)
        scaler = MinMaxScaler()

        # Combine transformers in ColumnTransformer
        transformers = []
        if label_encoding_columns:
            transformers.append(('label_encoder', label_encoder, label_encoding_columns))
        if onehot_encoding_columns:
            transformers.append(('onehot_encoder', onehot_encoder, onehot_encoding_columns))
        if scaling_columns:
            transformers.append(('scaler', scaler, scaling_columns))
        
        preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')

        # Create pipeline
        data_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])


        # Save the pipeline
        # save_object(self.data_preprocessing_config.preprocessed_object_file_path, data_pipeline)

        return data_pipeline
        # return label_encoding_function, onehot_encoding_function, scaling_function

    except Exception as e:
        raise BaseException(f"Error in get_data_preprocessor_object: {str(e)}", sys) from e
    